In [0]:
# Imports 

%tensorflow_version 2.x
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.python import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator,load_img 
from tensorflow.python.keras.layers import Dense,Conv2D, MaxPooling2D ,Activation, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D

In [0]:
# Model Configuration
img_width, img_height = 224, 224  
train_data_dir = './ProgDataRandom/prog_data/training/'
validation_data_dir = './ProgDataRandom/prog_data/validation/'
test_data_dir = './ProgDataRandom/prog_data/testing/'
nb_train_samples = 8500*5
nb_validation_samples = 1000*5
nb_test_samples = 500*5
epochs = 50
batch_size = 100
reg=0.0001
learning_rate = 0.0001

In [0]:
# Setting up the image format/image shape
if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3)
 

In [6]:
# Created a model by using the "ResNet50" as basemodel 
# and also appended few more layers to the base model
base_model = ResNet50(weights ="imagenet",include_top= False,input_shape=input_shape )

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(512,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg), activation='relu')(x)
x = Dense(256,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg), activation='relu')(x)
x = Dense(128,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg), activation='relu')(x)
x = Dropout(.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(5, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = True

model.summary()

sgd = tf.keras.optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)

model.compile( loss= keras.losses.categorical_crossentropy, 
               optimizer = sgd, 
               metrics=['accuracy'])

94773248/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_____________________________

In [7]:
# The section is responsible for feeding the data during the training process
datagen = ImageDataGenerator( rescale = 1./255,
                              rotation_range=20,
                              zoom_range=0.15,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest")   
test = ImageDataGenerator(rescale=1./255)
  
train_generator = datagen.flow_from_directory(
                train_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                ) 

validation_generator = datagen.flow_from_directory( 
                validation_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                )
test_generator = test.flow_from_directory( 
                test_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                )


Found 42500 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
Found 2500 images belonging to 5 classes.


In [0]:
# Started the Training proces
from tensorflow.python.keras.callbacks import ModelCheckpoint

filepath="Resnet50_weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history= model.fit(
    train_generator, 
    steps_per_epoch = nb_train_samples // batch_size, 
    epochs = epochs,
    validation_data = validation_generator, 
    validation_steps = nb_validation_samples // batch_size,
    shuffle = True,callbacks=callbacks_list
    )


model.save_weights('./model_saved_full.h5',overwrite= True)

Epoch 1/50
425/425 [==============================] - ETA: 0s - loss: 0.6342 - accuracy: 0.8106
Epoch 00001: val_accuracy improved from -inf to 0.73660, saving model to Resnet50_weights_best.hdf5
425/425 [==============================] - 582s 1s/step - loss: 0.6342 - accuracy: 0.8106 - val_loss: 0.8318 - val_accuracy: 0.7366
Epoch 2/50
425/425 [==============================] - ETA: 0s - loss: 0.6248 - accuracy: 0.8176
Epoch 00002: val_accuracy improved from 0.73660 to 0.74560, saving model to Resnet50_weights_best.hdf5
425/425 [==============================] - 578s 1s/step - loss: 0.6248 - accuracy: 0.8176 - val_loss: 0.8133 - val_accuracy: 0.7456
Epoch 3/50
425/425 [==============================] - ETA: 0s - loss: 0.6254 - accuracy: 0.8143
Epoch 00003: val_accuracy did not improve from 0.74560
425/425 [==============================] - 577s 1s/step - loss: 0.6254 - accuracy: 0.8143 - val_loss: 0.8258 - val_accuracy: 0.7410
Epoch 4/50
425/425 [==============================] - ETA:

In [0]:
# The best trainned weights are stored in my GDrive as I was using colab for trainning purpose
# So, loading the best weight from my GDrive
model.load_weights("/content/drive/My Drive/Resnet50_weights_best_7804.hdf5")


In [9]:
# performing Evaluation

print('\n# Evaluate on test data')
results = model.evaluate(test_generator,batch_size = batch_size,steps = 2500 // batch_size, use_multiprocessing=True)
print('test loss, test acc:', results)


# Evaluate on test data
25/25 [==============================] - 10s 417ms/step - loss: 1.1089 - accuracy: 0.7308
test loss, test acc: [1.1088836193084717, 0.7307999730110168]


In [10]:
# Testing the model with the best weight loaded 
# Examines the Testing set/ makes prediction of the testing set

import numpy as np
import os
import cv2
from sklearn.metrics import classification_report, confusion_matrix
#Y_pred = model.predict(test_generator,batch_size = batch_size,steps = 2500 // batch_size, use_multiprocessing=True)
label=[]
clas=[]
test_data_dir = './ProgDataRandom/prog_data/testing/'
for j,i in enumerate(np.sort(os.listdir(test_data_dir))):
    p=os.path.join(test_data_dir,i)
    for image in os.listdir(p):

            inp = os.path.join(p,image)
            img = np.asarray(load_img(inp))/255
            img = np.reshape(img,[1,img_width,img_height,3])

            classes = np.argmax(model.predict(img))
            
            clas.append(classes)
            label.append(j)
    print(j," completed.")
          


0  completed.
1  completed.
2  completed.
3  completed.
4  completed.


In [0]:
print(label,clas,sep="\n")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [11]:
# Generates the Confusion Matrix and other Reports
print('Confusion Matrix')
print(confusion_matrix(label, clas))
print('Classification Report')
target_names = ['class_0','class_1','class_2','class_3','class_4']
print(classification_report(label, clas, target_names=target_names))

Confusion Matrix
[[392  66  37   3   2]
 [ 67 414  19   0   0]
 [104 102 262  27   5]
 [ 72  20 130 276   2]
 [  3   5   9   0 483]]
Classification Report
              precision    recall  f1-score   support

     class_0       0.61      0.78      0.69       500
     class_1       0.68      0.83      0.75       500
     class_2       0.57      0.52      0.55       500
     class_3       0.90      0.55      0.68       500
     class_4       0.98      0.97      0.97       500

    accuracy                           0.73      2500
   macro avg       0.75      0.73      0.73      2500
weighted avg       0.75      0.73      0.73      2500



In [ ]:
# Use to visualize the result graphically
f1=pyplot.figure()
pyplot.plot(history.history['accuracy'], label='train_acc')
pyplot.plot(history.history['val_accuracy'], label='val_acc')
f2=pyplot.figure()
pyplot.plot(history.history['loss'], label='train_loss')
pyplot.plot(history.history['val_loss'], label='val_loss')
pyplot.legend()
pyplot.show() 